In [1]:
import tensorflow as tf

tf.test.is_gpu_available()
print(tf.config.list_physical_devices('GPU'))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
[]


In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.layers import (Input, Activation, Add, Dense, Conv2D,
                                     GlobalAveragePooling2D, MaxPooling2D,
                                     Dropout, BatchNormalization)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import regularizers


KeyboardInterrupt: 

# Data Preprocessing

In [ ]:
# Define the path where our dataset is stored
dataset_path = r"C:\Users\User\Desktop\UMS\FYP\FYP 1\dataset\archive\garbage-dataset"

# Retrieve the names of all folders (representing trash types) within the dataset directory
garbage_types = os.listdir(dataset_path)

# Set to store unique image dimensions for the entire dataset
all_dimensions_set = set()

# Iterate over each trash type (folder) to process images
for garbage_type in garbage_types:
    folder_path = os.path.join(dataset_path, garbage_type)
    
    # Verify that the current item is a directory
    if os.path.isdir(folder_path):
        image_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'jpeg'))]
        
        # Display the count of images in the current folder
        num_images = len(image_files)
        print(f"{garbage_type} folder contains {num_images} images.")
        
        # Loop over each image to check its dimensions
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            with Image.open(image_path) as img:
                # Extract the width, height, and channels (color depth) of the image and add to the dimensions set
                width, height = img.size
                channels = len(img.getbands())
                all_dimensions_set.add((width, height, channels))
                
# Determine if all images in the entire dataset have the same dimensions 
if len(all_dimensions_set) == 1: 
    width, height, channel = all_dimensions_set.pop()
    print(f"\nAll images in the dataset have the same dimensions: {width}x{height} with {channels} color channels.")
else:
    print("\nThe images in the dataset have different dimensions or color channels.")

In [ ]:
# Iterate over each trash type (folder) to display images
for garbage_type in garbage_types:
    folder_path = os.path.join(dataset_path, garbage_type)
    
    # Verify that the current item is a directory
    if os.path.isdir(folder_path):
        image_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'jpeg'))]
        
        # Select the first 10 images
        image_files = image_files[:10]
        
        # Set up subplots
        fig, axs = plt.subplots(1, 10, figsize=(15, 2))
        
        for i, image_file in enumerate(image_files):
            image_path = os.path.join(folder_path, image_file)
            with Image.open(image_path) as img:
                axs[i].imshow(img)
                axs[i].axis('off')
        
        plt.tight_layout()
        fig.suptitle(garbage_type, fontsize=20, y=1.03)
        plt.show()

In [ ]:
# Initialize an empty list to store image file paths and their respective labels
data = []

# Loop through each garbage type and collect its images' file paths
for garbage_type in garbage_types:
    for file in os.listdir(os.path.join(dataset_path, garbage_type)):
        # Append the image file path and its trash type (as a label) to the data list
        data.append((os.path.join(dataset_path, garbage_type, file), garbage_type))

# Convert the collected data into a DataFrame
df = pd.DataFrame(data, columns=['filepath', 'label'])

# Display the first few entries of the DataFrame
df.head()

In [ ]:
# Split with stratification
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Print the number of images in each set
print(f"Number of images in the training set: {len(train_df)}")
print(f"Number of images in the validation set: {len(val_df)}")

In [ ]:
# 1. Class distribution in the entire dataset
overall_distribution = df['label'].value_counts(normalize=True) * 100

# 2. Class distribution in the training set
train_distribution = train_df['label'].value_counts(normalize=True) * 100

# 3. Class distribution in the validation set
val_distribution = val_df['label'].value_counts(normalize=True) * 100

print("Class distribution in the entire dataset:\n")
print(overall_distribution.round(2))
print('-'*40)

print("\nClass distribution in the training set:\n")
print(train_distribution.round(2))
print('-'*40)

print("\nClass distribution in the validation set:\n")
print(val_distribution.round(2))

In [ ]:
import matplotlib.pyplot as plt

# Compute total number of images per class in the entire dataset
overall_count = df['label'].value_counts()

# Plot the bar chart
plt.figure(figsize=(10, 5))
plt.bar(overall_count.index, overall_count, color='blue', alpha=0.7)

# Formatting the plot
plt.xlabel("Class Labels")
plt.ylabel("Number of Images")
plt.title("Total Number of Images per Class in Dataset")
plt.xticks(rotation=45)  # Rotate labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Show the values on top of each bar
for i, v in enumerate(overall_count):
    plt.text(i, v + 5, str(v), ha='center', fontsize=10)

# Show the plot
plt.show()


In [ ]:
# Slight Augmentation settings for training
train_datagen = ImageDataGenerator(
    rescale=1./255,                     # Normalize pixel values to [0,1]
    rotation_range=45,                  # Randomly rotate the images by up to 45 degrees
    width_shift_range=0.15,             # Randomly shift images horizontally by up to 15% of the width
    height_shift_range=0.15,            # Randomly shift images vertically by up to 15% of the height
    zoom_range=0.15,                    # Randomly zoom in or out by up to 15%
    horizontal_flip=True,               # Randomly flip images horizontally
    vertical_flip=True,                 # Randomly flip images vertically
    shear_range=0.05,                   # Apply slight shear transformations
    brightness_range=[0.9, 1.1],        # Vary brightness between 90% to 110% of original
    channel_shift_range=10,             # Randomly shift channels (can change colors of images slightly but less aggressively)
    fill_mode='nearest'                 # Fill in missing pixels using the nearest filled value
)

# Only rescaling for validation
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
num_types = len(garbage_types)

# Set up the plot
fig, axs = plt.subplots(num_types, 3, figsize=(15, 5 * num_types))

for i, garbage_type in enumerate(garbage_types):
    folder_path = os.path.join(dataset_path, garbage_type)
    
    # Verify that the current item is a directory
    if os.path.isdir(folder_path):
        image_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'jpeg'))]
        
        if not image_files:
            continue
        
        # Select one image
        image_file = image_files[0]
        
        image_path = os.path.join(folder_path, image_file)
        sample_image = load_img(image_path)
        sample_image_array = img_to_array(sample_image)
        sample_image_array = np.expand_dims(sample_image_array, axis=0)  # Expand dims to match (1, height, width, channels)

        # Generate a batch of augmented images
        augmented_images = train_datagen.flow(sample_image_array, batch_size=1)
        
        # Display folder name on the left side
        axs[i, 0].text(0.5, 0.5, garbage_type, fontsize=18, ha='center', va='center')
        axs[i, 0].axis('off')
        
        # Display original image in the first column
        axs[i, 1].imshow(sample_image_array[0].astype('uint8'))
        axs[i, 1].axis('off')
        axs[i, 1].set_title('Original')
        
        # Display augmented image in the second column
        batch = next(augmented_images)
        image = batch[0]  # Take the first (and only) image in the batch
        axs[i, 2].imshow(image)
        axs[i, 2].axis('off')
        axs[i, 2].set_title('Augmented')

# Adjust layout and display
plt.tight_layout()
plt.show()

In [ ]:
# Using flow_from_dataframe to generate batches
# Generate training batches from the training dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,                  # DataFrame containing training data
    x_col="filepath",                    # Column with paths to image files
    y_col="label",                       # Column with image labels
    target_size=(224, 224),              # Resize all images to size of 224x224
    batch_size=4,                       # Number of images per batch
    class_mode='categorical',            # One-hot encode labels
    seed=42,                             # Seed for random number generator to ensure reproducibility
    shuffle=False                        # Data is not shuffled; order retained from DataFrame
)


# Generate validation batches from the validation dataframe
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,                    # DataFrame containing validation data
    x_col="filepath",                    # Column with paths to image files
    y_col="label",                       # Column with image labels
    target_size=(224, 224),              # Resize all images to size of 224x224
    batch_size=32,                       # Number of images per batch
    class_mode='categorical',            # One-hot encode labels
    seed=42,                             # Seed for random number generator to ensure reproducibility
    shuffle=False                        # Data is not shuffled; order retained from DataFrame
)

In [ ]:
print(f"Number of batches in train_generator: {len(train_generator)}")
print(f"Number of batches in val_generator: {len(val_generator)}")

In [ ]:
# Function to count total number of images generated by a generator
def count_images_in_generator(generator):
    total_images = 0
    for _ in generator:
        total_images += generator.batch_size
        if generator.batch_index == 0:
            break  # To break out of infinite loop when all batches have been seen
    return total_images

# Count total augmented images
total_train_images = count_images_in_generator(train_generator)
total_val_images = count_images_in_generator(val_generator)

print(f"Total number of augmented images in training set: {total_train_images}")
print(f"Total number of augmented images in validation set: {total_val_images}")

In [ ]:
# Extract class labels from the 'label' column of train_df
class_labels = train_df['label'].unique()
class_labels

In [ ]:
train_generator.class_indices

In [ ]:
# Compute class weights
weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=train_df['label'])
weights

In [ ]:
# Convert the computed weights to a dictionary for passing to model training
class_weights = dict(zip(train_generator.class_indices.values(), weights))
class_weights

# Building Resnet50 from scratch

In [ ]:
def residual_block(X, kernel_size, filters, reduce=False, stride=2):
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. We will need this later to add back to the main path. 
    X_shortcut = X
    
    if reduce:
        # If we are to reduce the spatial size, apply a 1x1 CONV layer to the shortcut path
        X = Conv2D(filters = F1, 
                   kernel_size = (1, 1), 
                   strides = (stride, stride), 
                   padding = 'valid', 
                   kernel_initializer='he_normal')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
        
        X_shortcut = Conv2D(filters = F3, 
                            kernel_size = (1, 1), 
                            strides = (stride, stride), 
                            padding = 'valid', 
                            kernel_initializer='he_normal')(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
    else: 
        # First component of main path
        X = Conv2D(filters = F1, 
                   kernel_size = (1, 1), 
                   strides = (1, 1), 
                   padding = 'valid', 
                   kernel_initializer='he_normal')(X)
        X = BatchNormalization(axis = 3)(X)
        X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, 
               kernel_size = (kernel_size, kernel_size), 
               strides = (1, 1), 
               padding = 'same', 
               kernel_initializer='he_normal')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, 
               kernel_size = (1, 1), 
               strides = (1, 1), 
               padding = 'valid', 
               kernel_initializer='he_normal')(X)
    X = BatchNormalization(axis = 3)(X)

    # Final step: Add shortcut value to main path, and pass it through a ReLU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


In [ ]:
def ResNet_50(input_shape, classes):

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Block 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='he_normal')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Block 2
    X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
    X = residual_block(X, 3, [64, 64, 256])
    X = residual_block(X, 3, [64, 64, 256])

    # Block 3 
    X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])

    # Block 4 
    X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])

    # Block 5 
    X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
    X = residual_block(X, 3, [512, 512, 2048])
    X = residual_block(X, 3, [512, 512, 2048])

    # Global Average Pooling to reduce spatial dimensions
    X = GlobalAveragePooling2D()(X)
    
    # Fully Connected Layer for classification
    X = Dense(classes, activation='softmax')(X)
        
    # Create the model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
def Modified_ResNet50(input_shape, classes):

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='he_normal')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
    X = residual_block(X, 3, [64, 64, 256])
    X = residual_block(X, 3, [64, 64, 256])

    # Stage 3 
    X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])

    # Stage 4 
    X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])

    # Stage 5 
    X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
    X = residual_block(X, 3, [512, 512, 2048])
    X = residual_block(X, 3, [512, 512, 2048])

    # Global Average Pooling to reduce spatial dimensions
    X = GlobalAveragePooling2D()(X)
    
    # Add Dropout to prevent overfitting
    X = Dropout(0.5)(X)
    
    # Fully Connected Layer for classification
    X = Dense(classes, activation='softmax')(X)
        
    # Create the model
    model = Model(inputs = X_input, outputs = X, name='Modified_ResNet50')

    return model

In [ ]:
# Define the shape of the input images and number of classes
input_shape = (224, 224, 3)
num_classes = 5

# Initialize the modified ResNet50 model with the specified parameters
modified_resnet50_model = Modified_ResNet50(input_shape=input_shape, classes=num_classes)

In [ ]:
plot_model(modified_resnet50_model, show_shapes=True, show_layer_names=False, dpi=120)

In [ ]:
modified_resnet50_model.summary()

In [ ]:
modified_resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Add ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=0.00001)

# Add EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=50, restore_best_weights=True, verbose=1)

In [ ]:
print(f"Number of batches in train_generator: {len(train_generator)}")
print(f"Number of batches in val_generator: {len(val_generator)}")

In [ ]:
# Total number of epochs
num_epochs = 200 

# Train the model
history = modified_resnet50_model.fit(train_generator, 
                                      steps_per_epoch=len(train_generator), 
                                      epochs=num_epochs, 
                                      validation_data=val_generator, 
                                      validation_steps=len(val_generator),
                                      class_weight=class_weights,
                                      callbacks=[reduce_lr, early_stopping])

In [ ]:
def plot_learning_curves(history, start_epoch=5):

    # Convert the history.history dict to a pandas DataFrame
    df = pd.DataFrame(history.history)

    # Plot the curves from the specified epoch onwards
    df = df.iloc[start_epoch-1:]

    # Set the style of seaborn for better visualization
    sns.set(rc={'axes.facecolor': '#f0f0fc'}, style='darkgrid')

    # Plotting the learning curves
    plt.figure(figsize=(15,6))

    # Plotting the training and validation loss
    plt.subplot(1, 2, 1)
    sns.lineplot(x=df.index, y=df['loss'], color='royalblue', label='Train Loss')
    sns.lineplot(x=df.index, y=df['val_loss'], color='orangered', linestyle='--', label='Validation Loss')
    plt.title('Loss Evolution')

    # Plotting the training and validation accuracy
    plt.subplot(1, 2, 2)
    sns.lineplot(x=df.index, y=df['accuracy'], color='royalblue', label='Train Accuracy')
    sns.lineplot(x=df.index, y=df['val_accuracy'], color='orangered', linestyle='--', label='Validation Accuracy')
    plt.title('Accuracy Evolution')

    plt.show()

In [ ]:
plot_learning_curves(history)

In [ ]:
def evaluate_model_performance(model, val_generator, class_labels):
    """
    Evaluate the model's performance on the validation set and print the classification report.

    Parameters:
    - model: The trained model.
    - val_generator: Validation data generator.
    - class_labels: List of class names.
    
    Returns:
    - report: Classification report as a string.
    """
    
    # Getting all the true labels for the validation set
    true_labels = val_generator.classes

    # Get the class labels (names) from the generator
    class_labels = list(val_generator.class_indices.keys())

    # To get the predicted labels, we predict using the model  
    predictions = model.predict(val_generator, steps=len(val_generator))
    
    # Take the argmax to get the predicted class indices.
    predicted_labels = np.argmax(predictions, axis=1)
    
    # Extracting true labels from the validation generator
    true_labels = val_generator.classes

    # Classification report
    report = classification_report(true_labels, predicted_labels, target_names=class_labels)
    print(report)
    print('\n')
    
    # Define a custom colormap
    colors = ["white", "royalblue"]
    cmap_cm = LinearSegmentedColormap.from_list("cmap_cm", colors)

    # Confusion Matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Plotting confusion matrix using seaborn
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, cmap=cmap_cm, fmt='d', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
evaluate_model_performance(modified_resnet50_model, val_generator, class_labels)

In [ ]:
# Load the ResNet50 model with weights pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
base_model.summary()

In [ ]:
len(base_model.layers) 

In [ ]:
for i, layer in enumerate(base_model.layers):
    if 140 <= i <= 150:
        print(i, layer.name)

In [ ]:
# Freeze the layers up to conv4_block6_out
for layer in base_model.layers[:143]: # include the layer 142
    layer.trainable = False

In [ ]:
# Create the new model with transfer learning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(5, activation='softmax')(x)

transfer_resnet50_model = Model(inputs=base_model.input, outputs=x)

# Compile the model
transfer_resnet50_model.compile(
    optimizer=Adam(learning_rate=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [ ]:
plot_model(transfer_resnet50_model, show_shapes=True, show_layer_names=False, dpi=120)

In [ ]:
transfer_resnet50_model.summary()

In [ ]:
# Slight Augmentation settings for training
train_datagen = ImageDataGenerator(
    rotation_range=60,                  # Randomly rotate the images by up to 60 degrees
    width_shift_range=0.15,             # Randomly shift images horizontally by up to 15% of the width
    height_shift_range=0.15,            # Randomly shift images vertically by up to 15% of the height
    zoom_range=0.20,                    # Randomly zoom in or out by up to 20%
    horizontal_flip=True,               # Randomly flip images horizontally
    vertical_flip=True,                 # Randomly flip images vertically
    shear_range=0.05,                   # Apply slight shear transformations
    brightness_range=[0.9, 1.1],        # Vary brightness between 90% to 110% of original
    channel_shift_range=10,             # Randomly shift channels (can change colors of images slightly but less aggressively)
    fill_mode='nearest',                 # Fill in missing pixels using the nearest filled value
    preprocessing_function=preprocess_input  # Add this line
)

# For the validation set, you might not have augmentation:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Add this line

In [ ]:
# Using flow_from_dataframe to generate batches
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,                  # DataFrame containing training data
    x_col="filepath",                    # Column with paths to image files
    y_col="label",                       # Column with image labels
    target_size=(224, 224),              # Resize all images to size of 224x224
    batch_size=32,                       # Number of images per batch
    class_mode='categorical',            # One-hot encode labels
    seed=42,                             # Seed for random number generator to ensure reproducibility
    shuffle=False                        # Data is not shuffled; order retained from DataFrame
)


# Generate validation batches from the validation dataframe
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,                    # DataFrame containing validation data
    x_col="filepath",                    # Column with paths to image files
    y_col="label",                       # Column with image labels
    target_size=(224, 224),              # Resize all images to size of 224x224
    batch_size=32,                       # Number of images per batch
    class_mode='categorical',            # One-hot encode labels
    seed=42,                             # Seed for random number generator to ensure reproducibility
    shuffle=False                        # Data is not shuffled; order retained from DataFrame
)

In [ ]:
# Define the callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True, verbose=1)

# Train the baseline transfer_resnet50_model
baseline_history = transfer_resnet50_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=50,
    class_weight=class_weights,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Save baseline metrics
baseline_val_loss = min(baseline_history.history['val_loss'])
baseline_val_accuracy = max(baseline_history.history['val_accuracy'])
print(f"Baseline Model - Val Loss: {baseline_val_loss}, Val Accuracy: {baseline_val_accuracy}")


In [ ]:
plot_learning_curves(baseline_history, start_epoch=1)

In [ ]:
evaluate_model_performance(transfer_resnet50_model, val_generator, class_labels)

In [ ]:
# Load your custom model (Replace with your model loading code)
model = transfer_resnet50_model

# Function to load and preprocess a random image from a folder
def load_random_image_from_folder(folder_path):
    image_files = [
        f for f in os.listdir(folder_path)
        if f.endswith(('jpg', 'jpeg', 'png'))
    ]
    if not image_files:
        raise FileNotFoundError(f"No valid image files in folder: {folder_path}")
    
    random_image_file = random.choice(image_files)
    image_path = os.path.join(folder_path, random_image_file)
    
    img = Image.open(image_path).convert('RGB')
    img_resized = img.resize((224, 224))
    img_array = img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    return img, img_array

# Function to predict and visualize one image per class
def predict_and_plot_each_class(dataset_path):
    class_labels = sorted([
        folder for folder in os.listdir(dataset_path)
        if os.path.isdir(os.path.join(dataset_path, folder))
    ])
    
    num_classes = len(class_labels)
    if num_classes == 0:
        raise ValueError("No class folders found in the dataset path.")
    
    fig, axs = plt.subplots(1, num_classes, figsize=(15, 5))
    fig.suptitle("Prediction Results for One Image Per Class", fontsize=16)
    
    for i, class_label in enumerate(class_labels):
        folder_path = os.path.join(dataset_path, class_label)
        
        try:
            img, img_array = load_random_image_from_folder(folder_path)
            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions, axis=1)[0]
            predicted_label = class_labels[predicted_class_index]
            predicted_confidence = predictions[0][predicted_class_index]
            
            axs[i].imshow(img)
            axs[i].axis('off')
            axs[i].set_title(
                f"Predicted: {predicted_label}\n"
                f"Confidence: {predicted_confidence*100:.2f}%\n"
                f"True Label: {class_label}"
            )
        except Exception as e:
            axs[i].axis('off')
            axs[i].set_title(f"Error: {e}")
    
    plt.tight_layout(rect=[0, 0, 1, 0.9])
    plt.show()

# Define dataset path
dataset_path = r"C:\Users\User\Desktop\UMS\FYP\FYP 1\dataset\TrashType_Image_Dataset"

# Call the function
predict_and_plot_each_class(dataset_path)


In [ ]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np

# Define the search space
search_space = [
    Real(1e-5, 1e-2, name='learning_rate', prior='log-uniform'),
    Real(0.1, 0.6, name='dropout'),
    Integer(16, 64, name='batch_size'),
]

@use_named_args(search_space)
def objective(**params):
    learning_rate = params['learning_rate']
    dropout = params['dropout']
    batch_size = params['batch_size']
    
    # Create a new model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout)(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    # Callbacks
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
    
    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        validation_data=val_generator,
        validation_steps=len(val_generator),
        epochs=10,  # Fewer epochs for faster optimization
        batch_size=batch_size,
        class_weight=class_weights,
        callbacks=[reduce_lr, early_stopping],
        verbose=0  # Suppress detailed output for optimization
    )
    
    # Validation loss is the metric to minimize
    val_loss = min(history.history['val_loss'])
    return val_loss

# Perform Bayesian Optimization
result = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=30,  # Number of evaluations
    n_initial_points=5,  # Number of random initial points
    random_state=42
)

# Extract the best parameters
best_params = {dim.name: val for dim, val in zip(search_space, result.x)}
print("Best hyperparameters found:")
print(best_params)


In [ ]:
from skopt.plots import plot_convergence
plot_convergence(result)


In [ ]:
# Create the new model with transfer learning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.1)(x)
x = Dense(5, activation='softmax')(x)

optimized_resnet50_model = Model(inputs=base_model.input, outputs=x)

# Compile the model
optimized_resnet50_model.compile(
    optimizer=Adam(learning_rate=1e-05), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [ ]:
# Slight Augmentation settings for training
train_datagen = ImageDataGenerator(
    rotation_range=60,                  # Randomly rotate the images by up to 60 degrees
    width_shift_range=0.15,             # Randomly shift images horizontally by up to 15% of the width
    height_shift_range=0.15,            # Randomly shift images vertically by up to 15% of the height
    zoom_range=0.20,                    # Randomly zoom in or out by up to 20%
    horizontal_flip=True,               # Randomly flip images horizontally
    vertical_flip=True,                 # Randomly flip images vertically
    shear_range=0.05,                   # Apply slight shear transformations
    brightness_range=[0.9, 1.1],        # Vary brightness between 90% to 110% of original
    channel_shift_range=10,             # Randomly shift channels (can change colors of images slightly but less aggressively)
    fill_mode='nearest',                 # Fill in missing pixels using the nearest filled value
    preprocessing_function=preprocess_input  # Add this line
)

# For the validation set, you might not have augmentation:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Add this line

In [ ]:
# Using flow_from_dataframe to generate batches
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,                  # DataFrame containing training data
    x_col="filepath",                    # Column with paths to image files
    y_col="label",                       # Column with image labels
    target_size=(224, 224),              # Resize all images to size of 224x224
    batch_size=32,                       # Number of images per batch
    class_mode='categorical',            # One-hot encode labels
    seed=42,                             # Seed for random number generator to ensure reproducibility
    shuffle=False                        # Data is not shuffled; order retained from DataFrame
)


# Generate validation batches from the validation dataframe
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,                    # DataFrame containing validation data
    x_col="filepath",                    # Column with paths to image files
    y_col="label",                       # Column with image labels
    target_size=(224, 224),              # Resize all images to size of 224x224
    batch_size=32,                       # Number of images per batch
    class_mode='categorical',            # One-hot encode labels
    seed=42,                             # Seed for random number generator to ensure reproducibility
    shuffle=False                        # Data is not shuffled; order retained from DataFrame
)

In [ ]:
# Define the callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True, verbose=1)

# Total number of epochs
num_epochs = 50  

# Train the model
history = optimized_resnet50_model.fit(train_generator,
                                      steps_per_epoch=len(train_generator), 
                                      epochs=num_epochs,
                                      validation_data=val_generator, 
                                      validation_steps=len(val_generator),
                                      class_weight=class_weights,
                                       batch_size=16,
                                      callbacks=[reduce_lr, early_stopping])

In [ ]:
plot_learning_curves(history, start_epoch=1)

In [ ]:
evaluate_model_performance(optimized_resnet50_model, val_generator, class_labels)

In [ ]:
# Load your custom model (Replace with your model loading code)
model = optimized_resnet50_model

# Function to load and preprocess a random image from a folder
def load_random_image_from_folder(folder_path):
    image_files = [
        f for f in os.listdir(folder_path)
        if f.endswith(('jpg', 'jpeg', 'png'))
    ]
    if not image_files:
        raise FileNotFoundError(f"No valid image files in folder: {folder_path}")
    
    random_image_file = random.choice(image_files)
    image_path = os.path.join(folder_path, random_image_file)
    
    img = Image.open(image_path).convert('RGB')
    img_resized = img.resize((224, 224))
    img_array = img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    return img, img_array

# Function to predict and visualize one image per class
def predict_and_plot_each_class(dataset_path):
    class_labels = sorted([
        folder for folder in os.listdir(dataset_path)
        if os.path.isdir(os.path.join(dataset_path, folder))
    ])
    
    num_classes = len(class_labels)
    if num_classes == 0:
        raise ValueError("No class folders found in the dataset path.")
    
    fig, axs = plt.subplots(1, num_classes, figsize=(15, 5))
    fig.suptitle("Prediction Results for One Image Per Class", fontsize=16)
    
    for i, class_label in enumerate(class_labels):
        folder_path = os.path.join(dataset_path, class_label)
        
        try:
            img, img_array = load_random_image_from_folder(folder_path)
            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions, axis=1)[0]
            predicted_label = class_labels[predicted_class_index]
            predicted_confidence = predictions[0][predicted_class_index]
            
            axs[i].imshow(img)
            axs[i].axis('off')
            axs[i].set_title(
                f"Predicted: {predicted_label}\n"
                f"Confidence: {predicted_confidence*100:.2f}%\n"
                f"True Label: {class_label}"
            )
        except Exception as e:
            axs[i].axis('off')
            axs[i].set_title(f"Error: {e}")
    
    plt.tight_layout(rect=[0, 0, 1, 0.9])
    plt.show()

# Define dataset path
dataset_path = r"C:\Users\User\Desktop\UMS\FYP\FYP 1\dataset\TrashType_Image_Dataset"

# Call the function
predict_and_plot_each_class(dataset_path)


In [ ]:
# Save optimized metrics
optimized_val_loss = min(history.history['val_loss'])
optimized_val_accuracy = max(history.history['val_accuracy'])
print(f"Optimized Model - Val Loss: {optimized_val_loss}, Val Accuracy: {optimized_val_accuracy}")

In [ ]:
# Determine the number of epochs for each model
baseline_epochs = len(baseline_val_loss)
optimized_epochs = len(optimized_val_loss)

# Use the shorter epoch count to avoid mismatch
min_epochs = min(baseline_epochs, optimized_epochs)
epochs = range(1, min_epochs + 1)

# Truncate validation loss and accuracy to match the shorter history
baseline_val_loss = baseline_val_loss[:min_epochs]
baseline_val_accuracy = baseline_val_accuracy[:min_epochs]
optimized_val_loss = optimized_val_loss[:min_epochs]
optimized_val_accuracy = optimized_val_accuracy[:min_epochs]

# Plot Validation Loss
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, baseline_val_loss, label='Baseline - Val Loss', color='dodgerblue', marker='o')
plt.plot(epochs, optimized_val_loss, label='Optimized - Val Loss', color='blue', marker='o')
plt.title('Validation Loss Across Epochs')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True)

# Plot Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, baseline_val_accuracy, label='Baseline - Val Accuracy', color='dodgerblue', marker='o')
plt.plot(epochs, optimized_val_accuracy, label='Optimized - Val Accuracy', color='blue', marker='o')
plt.title('Validation Accuracy Across Epochs')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Evaluate both models
optimized_resnet50_results = evaluate_model_performance(optimized_resnet50_model, val_generator, class_labels)
transfer_resnet50_results = evaluate_model_performance(transfer_resnet50_model, val_generator, class_labels)

# Extract accuracy from results (Assuming results are in the form of a dictionary)
optimized_resnet50_accuracy = optimized_resnet50_results['accuracy']  # Adjust if your results are different
transfer_resnet50_accuracy = transfer_resnet50_results['accuracy']  # Adjust if your results are different


In [ ]:
import matplotlib.pyplot as plt

# Create a bar plot to compare the accuracies
model_names = ['Optimized ResNet50', 'ResNet50']
accuracies = [optimized_resnet50_accuracy, transfer_resnet50_accuracy]

plt.bar(model_names, accuracies, color=['blue', 'green'])
plt.title('Accuracy Comparison: Optimized vs Transfer ResNet50')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # Assuming accuracy is between 0 and 1
plt.show()
